In [1]:
import pandas as pd

### Read in the Engagement File and Clean it Up

In [2]:
# Read in the file and look at the first five rows
engagement = pd.read_table('takehome_user_engagement.csv',sep=',')
print(engagement.shape)
engagement.head()

(207917, 3)


,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [9]:
# Count the number of null values in each column.
engagement.isnull().sum()

time_stamp    0
user_id       0
visited       0
dtype: int64

In [8]:
# Count the number of unique values in each column
engagement.apply(lambda x: len(x.unique()))

time_stamp    207220
user_id         8823
visited            1
dtype: int64

In [10]:
# The 'visited' column is all '1', so we can drop it.
engagement.drop(['visited'], axis=1, inplace = True)

In [5]:
# Type of value for each column
engagement.dtypes

time_stamp    object
user_id        int64
visited        int64
dtype: object

In [11]:
# Change time_stamp column to datetime format
# https://stackoverflow.com/questions/15891038/change-data-type-of-columns-in-pandas
engagement['time_stamp'] = pd.to_datetime(engagement['time_stamp'])

In [12]:
engagement.head()

,time_stamp,user_id
0,2014-04-22 03:53:30,1
1,2013-11-15 03:45:04,2
2,2013-11-29 03:45:04,2
3,2013-12-09 03:45:04,2
4,2013-12-25 03:45:04,2


### Adopted Users - "a user who has logged into the product on three separate days in at least one seven-day period"

To Mark Users as Adopted or Not, I will...
1. Count visits per day by user
2. Verify there are no multiple visits/day
3. Use timedelta to calculate length of time between visits
4. Check if <= 7

In [30]:
from datetime import datetime, timedelta

def userStatus(x):
    if len(x) >= 3:  
        x = [i for i in x]  # convert property object to list
        x.sort()
        x = [x[i+1] - x[i] for i in range(len(x)-2)]  # cumulative difference of current and next day
        x = [1 for i in range(len(x)-2) if x[i] + x[i+1] + x[i+2] <= timedelta(days=7)]
        if 1 in x:
            return 1

In [38]:
# Apply to engagement dataframe
adopted = engagement.groupby('user_id').agg(userStatus)  # group by user_id and aggregate using function defined above
adopted.fillna(0, inplace=True)  # fill null values with 0
adopted.columns = ['adopted_user']
adopted['user_id'] = adopted.index
print(adopted.shape) #verify all users are accounted for - should be same number as above
adopted.head()

(8823, 2)


,adopted_user,user_id
user_id,,
1,0.0,1
2,0.0,2
3,0.0,3
4,0.0,4
5,0.0,5


### Read in Users File and Clean it Up

In [40]:
# Read in the file and look at the first five rows
users = pd.read_table('takehome_users.csv',sep=',')
print(users.shape)
users.head()

(12000, 10)


,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [41]:
# Type of value for each column
users.dtypes

object_id                       int64
creation_time                  object
name                           object
email                          object
creation_source                object
last_session_creation_time    float64
opted_in_to_mailing_list        int64
enabled_for_marketing_drip      int64
org_id                          int64
invited_by_user_id            float64
dtype: object

In [42]:
# Change time_stamp column to datetime format
# https://stackoverflow.com/questions/15891038/change-data-type-of-columns-in-pandas
users['creation_time'] = pd.to_datetime(users['creation_time'])
# https://www.laurivan.com/convert-timestamp-to-datetime-for-pandas-dataframe/
users['last_session_creation_time'] = pd.to_datetime(users['last_session_creation_time'], unit='s')

In [43]:
# Type of value for each column
users.dtypes

object_id                              int64
creation_time                 datetime64[ns]
name                                  object
email                                 object
creation_source                       object
last_session_creation_time    datetime64[ns]
opted_in_to_mailing_list               int64
enabled_for_marketing_drip             int64
org_id                                 int64
invited_by_user_id                   float64
dtype: object

In [44]:
# Count the number of unique values in each column
users.apply(lambda x: len(x.unique()))

object_id                     12000
creation_time                 11996
name                          11355
email                         11980
creation_source                   5
last_session_creation_time     8822
opted_in_to_mailing_list          2
enabled_for_marketing_drip        2
org_id                          417
invited_by_user_id             2565
dtype: int64

In [45]:
# We can drop the name and email columns. It's interesting that not all emails are unique. We will not take out any
#rows because of this. The behavior will be worked into the model.
users.drop(['name','email'], axis=1, inplace = True)

In [46]:
# Count the number of null values in each column.
users.isnull().sum()

object_id                        0
creation_time                    0
creation_source                  0
last_session_creation_time    3177
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id            5583
dtype: int64

In [47]:
# Fill '0' if not invited by anyone
users['invited_by_user_id'].fillna(0, inplace=True)
# Make last_session_creation_time creation_time if null
users['last_session_creation_time'].fillna(users['creation_time'], inplace=True)

In [48]:
# Add a column that shows the diff between the last time the column was used and when it was created. 0 would mean the
#user has not logged in since they created the account, and alternatively, a very large amount, would mean there was
#a lot of time between making the account and their last login.
users['time_diff'] = users['last_session_creation_time'] - users['creation_time']
users.head()

,object_id,creation_time,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,time_diff
0,1,2014-04-22 03:53:30,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0,0 days
1,2,2013-11-15 03:45:04,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,136 days
2,3,2013-03-19 23:14:52,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525.0,0 days
3,4,2013-05-21 08:09:28,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151.0,1 days
4,5,2013-01-17 10:14:20,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240.0,5 days


In [49]:
users.drop(['last_session_creation_time','creation_time'], axis=1, inplace = True)

In [50]:
users.rename(columns={'object_id': 'user_id'}, inplace=True)

### Merge the Two Datasets Together

In [64]:
df = pd.merge(users,adopted, how='left', on='user_id')
df.head()

,user_id,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,time_diff,adopted_user
0,1,GUEST_INVITE,1,0,11,10803.0,0 days,0.0
1,2,ORG_INVITE,0,0,1,316.0,136 days,0.0
2,3,ORG_INVITE,0,0,94,1525.0,0 days,0.0
3,4,GUEST_INVITE,0,0,1,5151.0,1 days,0.0
4,5,GUEST_INVITE,0,0,193,5240.0,5 days,0.0


In [65]:
# Count the number of null values in each column.
df.isnull().sum()

user_id                          0
creation_source                  0
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id               0
time_diff                        0
adopted_user                  3177
dtype: int64

In [66]:
# Fill adopted user with zero - assume they are not an adopted user
df['adopted_user'].fillna(0, inplace=True)

In [67]:
df.dtypes

user_id                                 int64
creation_source                        object
opted_in_to_mailing_list                int64
enabled_for_marketing_drip              int64
org_id                                  int64
invited_by_user_id                    float64
time_diff                     timedelta64[ns]
adopted_user                          float64
dtype: object

In [68]:
# Convert time_diff to number
df['time_diff'] = df['time_diff'].map(lambda df: df.total_seconds())

In [69]:
# Create dummies for creation_source
df = pd.get_dummies(df,columns=['creation_source'])
df.head()

,user_id,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,time_diff,adopted_user,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
0,1,1,0,11,10803.0,0.0,0.0,1,0,0,0,0
1,2,0,0,1,316.0,11750400.0,0.0,0,1,0,0,0
2,3,0,0,94,1525.0,0.0,0.0,0,1,0,0,0
3,4,0,0,1,5151.0,86400.0,0.0,1,0,0,0,0
4,5,0,0,193,5240.0,432000.0,0.0,1,0,0,0,0


## Set Up the Predictive Model

In [70]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn import decomposition
from sklearn.preprocessing import MinMaxScaler

In [71]:
# Target variable
y = df['adopted_user']

In [72]:
# Select features to include in model. Will leave out 'invited_by_user_id' because it would make too many features
x = df[['opted_in_to_mailing_list','enabled_for_marketing_drip','org_id','time_diff',
        'creation_source_GUEST_INVITE','creation_source_ORG_INVITE','creation_source_PERSONAL_PROJECTS',
        'creation_source_SIGNUP','creation_source_SIGNUP_GOOGLE_AUTH']]

In [73]:
# Use MinMaxScaler to scale values to [0,1]
scaler = MinMaxScaler()
scaler.fit(x)
scaler.fit_transform(x)

array([[ 1.        ,  0.        ,  0.02644231, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.00240385, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.22596154, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 1.        ,  1.        ,  0.19951923, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.01442308, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  0.        ]])

### Train/Test Split

In [74]:
# 70% train and 30% test data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)

### Train the Model

In [75]:
tree = DecisionTreeClassifier()

tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### Test the Model

In [76]:
y_pred = tree.predict(X_test)

# scores
print('Feature importance')
for idx, val in enumerate(tree.feature_importances_):
    print("{:30}{:3f}".format(x.columns[idx], val))

Feature importance
opted_in_to_mailing_list      0.004769
enabled_for_marketing_drip    0.003252
org_id                        0.067244
time_diff                     0.905096
creation_source_GUEST_INVITE  0.002184
creation_source_ORG_INVITE    0.006898
creation_source_PERSONAL_PROJECTS0.001719
creation_source_SIGNUP        0.004346
creation_source_SIGNUP_GOOGLE_AUTH0.004493


## Conclusion

The difference in time between when a user last logged in and when they signed up seems to be a clear indicator of whehter or not a user will become an adopted user. The next most important feature is org_id. Users most likely use the service through work and do not have as much of a choice of whether or not to use.